<a href="https://colab.research.google.com/github/Angatang/University-Recommender/blob/Tree-Only/Tree_Only_Uni_Reco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from collections import Counter
from sklearn.metrics import precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix

In [ ]:
df = pd.read_excel("/content/Data_21-24 (1)-1.xlsx", index_col=0)

NameError: name 'pd' is not defined

In [ ]:
if 'Is Competitive Exam Given' in df.columns:
    print("Column 'Is Competitive Exam Given' exists in df.")
    print("Value Counts (including NaNs):")
    print(df['Is Competitive Exam Given'].value_counts(dropna=False))
    # Check data type as well
    print(f"Data type: {df['Is Competitive Exam Given'].dtype}")

    # Also, let's re-check the scores *before* they are potentially overwritten
    # Make sure the column name is correct for your raw data before conversion
    print("\nChecking original competitive scores in df_us (after potential to_numeric):")
    original_scores = pd.to_numeric(df['Total Score (Competitive Exam)'], errors='coerce')
    print(original_scores.value_counts(dropna=False))

else:
    print("ERROR: Column 'Is Competitive Exam Given' DOES NOT EXIST in df!")

In [ ]:
print("\nParsing College Details...")

def parse_college_details(details):
    if pd.isna(details):
        return pd.Series([np.nan] * 4) # Return NaNs if input is NaN

    # Regex to capture: Major, College, University, Year
    # Adjust pattern if format varies significantly
    # Using non-greedy matching (.*?) and being specific about keywords
    pattern = r"^(.*?),\s*College:\s*(.*?),\s*University:\s*(.*?),\s*Year:\s*(\d{4})$"
    match = re.search(pattern, str(details).strip())

    if match:
        ug_major = match.group(1).strip()
        ug_college = match.group(2).strip()
        ug_university = match.group(3).strip()
        ug_year = int(match.group(4)) # Convert year to integer
        return pd.Series([ug_major, ug_college, ug_university, ug_year])
    else:
        # Attempt simpler parsing if primary fails (e.g. if missing parts)
        # Fallback: return NaNs or try extracting at least year/major if possible
        # This part might need refinement based on actual variations in your data
        year_match = re.search(r"Year:\s*(\d{4})", str(details))
        ug_year = int(year_match.group(1)) if year_match else np.nan
        # Add more fallback logic if needed
        return pd.Series([np.nan] * 3 + [ug_year]) # Example fallback

# Apply the function to create new columns
parsed_cols = ['UG_Major', 'UG_College_Name', 'UG_University_Name', 'UG_Year_of_Graduation']
df[parsed_cols] = df['College Details'].apply(parse_college_details)


In [ ]:
print("\nCalculating Overall GPA on 4.0 Scale...")

# Define the list of score columns
score_columns = [
    'First Year Score', 'Second Year Score', 'Third Year Score',
    'Fourth Year Score', 'Fifth Year Score', 'Sixth Year Score',
    'Seventh Year Score', 'Eighth Year Score'
]

# Convert score columns to numeric, coercing errors to NaN
for col in score_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

def calculate_gpa_4_scale(row):
    system = row['Grading Systen']
    scores = row[score_columns].tolist()
    valid_scores = []

    if pd.isna(system):
        return np.nan

    # Determine scale and outlier bounds
    max_scale = np.nan
    lower_bound = np.nan
    upper_bound = np.nan

    if system == 'CGPA10':
        max_scale = 10.0
        lower_bound = 1.0 # Adjust if lower scores are valid
        upper_bound = 10.0
    elif system == 'GPA7':
        max_scale = 7.0
        lower_bound = 1.0 # Adjust if lower scores are valid
        upper_bound = 7.0
    elif system == 'Default': # Percentage scale
        max_scale = 100.0
        lower_bound = 35.0 # Assuming scores below 35% are outliers/fails to ignore
        upper_bound = 100.0
    else: # Handle unexpected systems
        return np.nan

    # Filter scores
    for score in scores:
        if pd.notna(score) and lower_bound <= score <= upper_bound:
             valid_scores.append(score)

    # Calculate average and convert
    if not valid_scores: # Check if list is empty
        return np.nan
    else:
        average_score = sum(valid_scores) / len(valid_scores)
        return (average_score / max_scale) * 4.0

# Apply the function row-wise
df['Overall_GPA_4_Scale'] = df.apply(calculate_gpa_4_scale, axis=1)


In [ ]:
if 'Is Competitive Exam Given' in df.columns:
    print("Column 'Is Competitive Exam Given' exists in df.")
    print("Value Counts (including NaNs):")
    print(df['Is Competitive Exam Given'].value_counts(dropna=False))
    # Check data type as well
    print(f"Data type: {df['Is Competitive Exam Given'].dtype}")

    # Also, let's re-check the scores *before* they are potentially overwritten
    # Make sure the column name is correct for your raw data before conversion
    print("\nChecking original competitive scores in df_us (after potential to_numeric):")
    original_scores = pd.to_numeric(df['Total Score (Competitive Exam)'], errors='coerce')
    print(original_scores.value_counts(dropna=False))

else:
    print("ERROR: Column 'Is Competitive Exam Given' DOES NOT EXIST in df!")

In [ ]:
print("\nHandling Test Scores")

# Define Max Scores
MAX_SCORES = {
    'GRE': 340,
    'GMAT': 800,  # Traditional GMAT Max
    'SAT': 1600,
    'ACT': 36
}

# --- Language Exam (No changes needed here from previous correction) ---
lang_score_col = 'Total Score (Language Exam)'
lang_given_col = 'Is Language Exam Given'
df[lang_score_col] = pd.to_numeric(df[lang_score_col], errors='coerce')
df[lang_given_col] = pd.to_numeric(df[lang_given_col], errors='coerce')
df.loc[(df[lang_given_col] == 0.0) | (df[lang_given_col].isna()), lang_score_col] = 0
df.loc[(df[lang_given_col] == 1.0) & (df[lang_score_col].isna()), lang_score_col] = -2

# --- Competitive Exam ---
comp_score_col = 'Total Score (Competitive Exam)'
comp_given_col = 'Is Competitive Exam Given'
comp_exam_type_col = 'Competitive Exam' # Column specifying GRE/GMAT/SAT/ACT

# Ensure score column is numeric
df[comp_score_col] = pd.to_numeric(df[comp_score_col], errors='coerce')
# Ensure 'Is Given' column is numeric/float
df[comp_given_col] = pd.to_numeric(df[comp_given_col], errors='coerce')

# Apply logic based on the 'Is Given' flag (1.0 = Yes, 0.0 = No, NaN = Treat as No)
df.loc[(df[comp_given_col] == 0.0) | (df[comp_given_col].isna()), comp_score_col] = 0
df.loc[(df[comp_given_col] == 1.0) & (df[comp_score_col].isna()), comp_score_col] = -2

# !!! NEW CAPPING LOGIC !!!
# Ensure the Exam Type column is string and handle potential NaNs
df[comp_exam_type_col] = df[comp_exam_type_col].astype(str).fillna('UNKNOWN')

print("Applying score capping...")
for exam_type, max_score in MAX_SCORES.items():
    # Condition for rows matching the exam type AND score exceeding max
    condition = (
        (df[comp_exam_type_col].str.upper() == exam_type) &
        (df[comp_score_col] > max_score)
    )
    # Apply capping
    count_capped = condition.sum()
    if count_capped > 0:
        print(f"Capping {count_capped} scores for {exam_type} at {max_score}")
        df.loc[condition, comp_score_col] = max_score

# --- Verification ---
print("Sample processed test scores (Corrected + Capped):")
print(df[[lang_given_col, lang_score_col, comp_given_col, comp_score_col, comp_exam_type_col]].head())
print("\nValue counts for Competitive Score after correction + capping:")
# Check if capping worked by looking at max values per exam type if needed
# print(df_us.groupby(comp_exam_type_col)[comp_score_col].max())
print(df[comp_score_col].value_counts(dropna=False))

In [ ]:
df = df.reset_index().rename(columns={'index': 'StudentID'}) # Use index as StudentID

print("\nColumns PRESENT in 'df' AFTER reset_index/rename:")
print(df.columns.tolist())
# -----------------------------------------

# --- Define Feature Columns (Adjust based on actual cleaned df) ---
print("\nDefining feature columns list...")
student_features_to_keep = [
    'Overall_GPA_4_Scale',
    'Total Score (Language Exam)', # Use cleaned/capped scores
    'Total Score (Competitive Exam)', # Use cleaned/capped scores
    'Language Exam',
    'Competitive Exam',
    'Future Desired Degree',
    'Desired Field of Study',
    'Desired Specialization'
]
required_cols_check = student_features_to_keep + ['Suggested Universities', 'StudentID']

# --- Check for required columns (This is where the error occurs) ---
print("\nChecking if required columns exist...")
missing_cols_check = [col for col in required_cols_check if col not in df.columns]

if missing_cols_check:
    print(f"DEBUG: Columns currently in df before raising error: {df.columns.tolist()}") # Add another check just before error
    raise ValueError(f"Missing required columns after initial cleaning: {missing_cols_check}")
else:
    print("All required columns found.")


# --- This part will only run if the check passes ---
df_for_restructure = df[required_cols_check].copy() # Keep only needed columns
print("Successfully created df_for_restructure.")

In [ ]:
# --- Parsing Function ---
def parse_suggestions(text):
    suggestions = []
    if isinstance(text, str):
        lines = text.split('\n')
        for line in lines:
            parts = line.split('|')
            if len(parts) >= 2:
                uni_name = parts[0].strip()
                status = parts[1].strip().upper()
                if uni_name:
                    suggestions.append({'UniversityName': uni_name, 'Status': status})
    return suggestions


In [ ]:
# --- Generate Positive and Potential Negative Examples ---
print("Parsing suggestions and generating examples...")
all_examples_rows = []
positive_statuses = ['ADMITRECEIVED', 'FINALIZED', 'ACCEPTED'] # Define positive statuses

for index, student_row in df_for_restructure.iterrows():
    student_id = student_row['StudentID']
    suggestions_text = student_row['Suggested Universities']
    parsed_list = parse_suggestions(suggestions_text)
    if not parsed_list: continue

    student_features = student_row[student_features_to_keep].to_dict()

    for suggestion in parsed_list:
        example_row = student_features.copy()
        example_row['StudentID'] = student_id
        example_row['UniversityName'] = suggestion['UniversityName']
        status = suggestion['Status']
        example_row['Admitted'] = 1 if status in positive_statuses else 0
        all_examples_rows.append(example_row)

if not all_examples_rows:
     raise ValueError("No examples generated. Check 'Suggested Universities' data/parsing.")

all_examples_df = pd.DataFrame(all_examples_rows)
print(f"Generated {all_examples_df.shape[0]} raw examples.")


In [ ]:
# --- Ensure Consistency (Prioritize Positive) ---
print("Resolving conflicts...")
all_examples_df['Student_Uni_Key'] = all_examples_df['StudentID'].astype(str) + "||" + all_examples_df['UniversityName']
positive_df = all_examples_df[all_examples_df['Admitted'] == 1].drop_duplicates(subset=['Student_Uni_Key'])
negative_df = all_examples_df[all_examples_df['Admitted'] == 0]
negative_df = negative_df[~negative_df['Student_Uni_Key'].isin(set(positive_df['Student_Uni_Key']))].drop_duplicates(subset=['Student_Uni_Key'])
interaction_df = pd.concat([positive_df, negative_df], ignore_index=True)
interaction_df = interaction_df.drop(columns=['Student_Uni_Key'])
print(f"Interaction DataFrame shape after de-duplication: {interaction_df.shape}")
print("Final 'Admitted' distribution:")
print(interaction_df['Admitted'].value_counts(normalize=True))


In [ ]:
# --- Final Feature Selection & NaN Handling ---
print("Selecting features and handling NaNs...")
# Define feature types (UniversityName is now categorical)
numerical_features = ['Overall_GPA_4_Scale', 'Total Score (Language Exam)', 'Total Score (Competitive Exam)']
categorical_features = [
    'Language Exam', 'Competitive Exam', 'Future Desired Degree',
    'Desired Field of Study', 'Desired Specialization', 'UniversityName'
]
features_to_use = numerical_features + categorical_features
# Ensure columns exist
available_features = [col for col in features_to_use if col in interaction_df.columns]
interaction_df_final = interaction_df[available_features + ['Admitted']].copy()

# Fill NaNs
for col in numerical_features:
    if col in interaction_df_final.columns:
        interaction_df_final[col] = interaction_df_final[col].fillna(interaction_df_final[col].median()) # Use median for scores
for col in categorical_features:
     if col in interaction_df_final.columns:
        interaction_df_final[col] = interaction_df_final[col].fillna('Unknown').astype(str)

X = interaction_df_final.drop('Admitted', axis=1)
y = interaction_df_final['Admitted']


In [ ]:
# --- Train-Test Split ---
print("Splitting data into Training (80%) and Testing (20%) sets...")
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.20, # Held-out test set
    random_state=42,
    stratify=y
)
print(f"Training set shape: X_train {X_train.shape}, y_train {y_train.shape}")
print(f"Testing set shape: X_test {X_test.shape}, y_test {y_test.shape}")


# --- Data Restructuring Complete ---
print("\nData restructured. Ready for preprocessing pipeline and hyperparameter tuning.")


In [ ]:
# Phase 2: Defining Preprocessor & Tuning Models

# --- 1. Define Preprocessing Pipeline ---
# Re-identify numerical and categorical columns IN X_train (in case some were dropped)
current_numerical = [col for col in numerical_features if col in X_train.columns]
current_categorical = [col for col in categorical_features if col in X_train.columns]

# WARNING: OneHotEncoding 'UniversityName' might create VERY many features.
# Consider HashingVectorizer(n_features=...) or Target Encoding for UniversityName
# if dimensionality becomes problematic. For now, using OHE.
preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), current_numerical),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), current_categorical)
    ],
    remainder='drop' # Drop any columns not specified (like StudentID if it remained)
)

In [ ]:
# --- 2. Define Parameter Grids ---
# Reduced grids for faster demonstration - expand ranges for thorough search
param_grid_rf = {
    'classifier__n_estimators': [100, 200],
    'classifier__max_depth': [10, 20, None],
    'classifier__min_samples_split': [2, 5],
    'classifier__class_weight': ['balanced', None] # Handle imbalance
}

param_grid_xgb = {
    'classifier__n_estimators': [1200],
    'classifier__max_depth': [75],
    'classifier__learning_rate': [0.4],
    'classifier__gamma': [0]
    # Add 'scale_pos_weight' if severe imbalance needs addressing
}

In [ ]:
# --- 3. Set up Pipelines and GridSearchCV ---
# Random Forest Pipeline
pipeline_rf = Pipeline(steps=[('preprocessor', preprocessor),
                            ('classifier', RandomForestClassifier(random_state=42, n_jobs=-1))])

# XGBoost Pipeline
pipeline_xgb = Pipeline(steps=[('preprocessor', preprocessor),
                             ('classifier', XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss'))])

# GridSearchCV Instances
search_rf = GridSearchCV(pipeline_rf, param_grid_rf, cv=3, scoring='roc_auc', n_jobs=-1, verbose=1)
search_xgb = GridSearchCV(pipeline_xgb, param_grid_xgb, cv=3, scoring='roc_auc', n_jobs=-1, verbose=1)


In [ ]:
# --- 4. Run Hyperparameter Tuning (on Training Data) ---
print("\nStarting GridSearchCV for Random Forest...")
search_rf.fit(X_train, y_train)
print(f"Best RF Params: {search_rf.best_params_}")
print(f"Best RF CV ROC-AUC: {search_rf.best_score_:.4f}")
best_rf_model = search_rf.best_estimator_ # Save the best pipeline

print("\nStarting GridSearchCV for XGBoost...")
search_xgb.fit(X_train, y_train)
print(f"Best XGB Params: {search_xgb.best_params_}")
print(f"Best XGB CV ROC-AUC: {search_xgb.best_score_:.4f}")
best_xgb_model = search_xgb.best_estimator_ # Save the best pipeline


In [ ]:
# --- 5. Evaluate Best Models on Test Set ---
print("\n--- Evaluating Best Models on Test Set ---")

# Random Forest Evaluation
print("\n--- Random Forest (Test Set) ---")
y_pred_rf_test = best_rf_model.predict(X_test)
y_prob_rf_test = best_rf_model.predict_proba(X_test)[:, 1]
print(f"Test ROC-AUC: {roc_auc_score(y_test, y_prob_rf_test):.4f}")
print("Classification Report:")
print(classification_report(y_test, y_pred_rf_test))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_rf_test))

# XGBoost Evaluation
print("\n--- XGBoost (Test Set) ---")
y_pred_xgb_test = best_xgb_model.predict(X_test)
y_prob_xgb_test = best_xgb_model.predict_proba(X_test)[:, 1]
print(f"Test ROC-AUC: {roc_auc_score(y_test, y_prob_xgb_test):.4f}")
print("Classification Report:")
print(classification_report(y_test, y_pred_xgb_test))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_xgb_test))

# --- Tuning and Evaluation Complete ---
print("\nHyperparameter tuning and test set evaluation complete.")

In [ ]:
def plot_cv_results(search_cv_object, param_x, param_hue, model_name):
    """
    Plots mean CV score against one hyperparameter, potentially hueing by another.
    Assumes search_cv_object is a fitted GridSearchCV instance.
    """
    cv_results = pd.DataFrame(search_cv_object.cv_results_)

    # Extract parameter names correctly (they are prefixed)
    param_x_col = f'param_classifier__{param_x}'
    param_hue_col = f'param_classifier__{param_hue}' if param_hue else None

    # Check if columns exist
    required_plot_cols = [param_x_col, 'mean_test_score']
    if param_hue_col:
        required_plot_cols.append(param_hue_col)
    if not all(col in cv_results.columns for col in required_plot_cols):
        print(f"Error: Not all required columns found in cv_results_ for plotting. Available: {cv_results.columns.tolist()}")
        print(f"Looking for: {required_plot_cols}")
        return

    plt.figure(figsize=(10, 6))
    sns.lineplot(
        data=cv_results,
        x=param_x_col,
        y='mean_test_score', # Mean score on validation folds during CV
        hue=param_hue_col, # Optional: Color lines by another parameter
        marker='o',
        errorbar=('ci', 95) # Show confidence interval if multiple runs per point
    )
    plt.title(f'{model_name}: Mean ROC-AUC vs. {param_x}' + (f' (Grouped by {param_hue})' if param_hue else ''))
    plt.xlabel(param_x)
    plt.ylabel('Mean ROC-AUC (Cross-Validation)')
    plt.grid(True)
    if param_hue_col:
        plt.legend(title=param_hue, bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.tight_layout(rect=[0, 0, 0.85, 1])
    else:
        plt.tight_layout()
    plt.show()

In [ ]:
# --- Plot for Random Forest ---
if 'search_rf' in locals() and hasattr(search_rf, 'best_estimator_'):
    print("\n--- Visualizing Random Forest Tuning Results ---")
    # Example: Plot ROC-AUC vs. n_estimators, grouped by max_depth
    plot_cv_results(search_rf, param_x='n_estimators', param_hue='max_depth', model_name='Random Forest')
    # Example: Plot ROC-AUC vs. min_samples_split, grouped by class_weight
    plot_cv_results(search_rf, param_x='min_samples_split', param_hue='class_weight', model_name='Random Forest')
else:
    print("Cannot plot RF results: 'search_rf' not found or not fitted.")


In [ ]:
# --- Plot for XGBoost ---
if 'search_xgb' in locals() and hasattr(search_xgb, 'best_estimator_'):
    print("\n--- Visualizing XGBoost Tuning Results ---")
    # Example: Plot ROC-AUC vs. n_estimators, grouped by max_depth
    plot_cv_results(search_xgb, param_x='n_estimators', param_hue='max_depth', model_name='XGBoost')
    # Example: Plot ROC-AUC vs. learning_rate, grouped by gamma
    plot_cv_results(search_xgb, param_x='learning_rate', param_hue='gamma', model_name='XGBoost')
else:
    print("Cannot plot XGBoost results: 'search_xgb' not found or not fitted.")


In [ ]:
# Make sure the GridSearchCV objects 'search_rf' and 'search_xgb' exist and have been fitted.

print("\n--- Analyzing Feature Importances ---")

# --- Function to Get and Plot Importances ---
def plot_feature_importances(search_cv_object, X_train_cols, model_name, top_n=20):
    """
    Extracts, processes, and plots feature importances from a fitted
    GridSearchCV object containing a Pipeline(preprocessor, classifier).
    """
    print(f"\n--- {model_name} Feature Importances ---")
    try:
        # 1. Access the best pipeline found by GridSearchCV
        best_pipeline = search_cv_object.best_estimator_

        # 2. Access the fitted classifier step
        classifier = best_pipeline.named_steps['classifier']

        # 3. Access the fitted preprocessor step
        preprocessor_fitted = best_pipeline.named_steps['preprocessor']

        # 4. Get feature importances from the classifier
        importances = classifier.feature_importances_

        # 5. Get feature names AFTER preprocessing (including OHE columns)
        # Use the get_feature_names_out method of the fitted ColumnTransformer
        try:
            feature_names = preprocessor_fitted.get_feature_names_out(input_features=X_train_cols)
        except TypeError:
             # Older sklearn versions might not support input_features argument directly
             # Or try without input_features if it fails, though less reliable with complex transformers
             print("Warning: Using get_feature_names_out without input_features, names might be generic.")
             feature_names = preprocessor_fitted.get_feature_names_out()


        if len(importances) != len(feature_names):
            print(f"Error: Mismatch between number of importances ({len(importances)}) and feature names ({len(feature_names)}). Cannot plot.")
            return None

        # 6. Create a DataFrame for easier handling
        importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})

        # 7. Sort by importance and select top N
        importance_df = importance_df.sort_values(by='Importance', ascending=False).head(top_n)

        # 8. Print Top N Importances
        print(f"\nTop {top_n} Features:")
        print(importance_df)

        # 9. Plot Top N Importances
        plt.figure(figsize=(10, top_n / 2.5)) # Adjust figure size based on N
        sns.barplot(x='Importance', y='Feature', data=importance_df, palette='viridis')
        plt.title(f'Top {top_n} Feature Importances ({model_name})')
        plt.xlabel('Importance Score')
        plt.ylabel('Feature')
        plt.tight_layout()
        plt.show()

        return importance_df # Return the dataframe for potential further use

    except AttributeError as e:
        print(f"Error accessing attributes, likely model or preprocessor steps incorrect: {e}")
        print("Make sure GridSearchCV has run and 'best_estimator_' contains a Pipeline named 'preprocessor' and 'classifier'.")
        return None
    except Exception as e:
        print(f"An error occurred during importance calculation/plotting: {e}")
        return None

# --- Generate Importances for Random Forest ---
# Ensure X_train still holds the original columns before preprocessing within the pipeline
if 'search_rf' in locals() and hasattr(search_rf, 'best_estimator_'):
    plot_feature_importances(search_rf, X_train.columns, "Random Forest")
else:
    print("GridSearchCV object 'search_rf' not found or not fitted. Skipping RF importance plot.")


# --- Generate Importances for XGBoost ---
if 'search_xgb' in locals() and hasattr(search_xgb, 'best_estimator_'):
    plot_feature_importances(search_xgb, X_train.columns, "XGBoost")
else:
     print("GridSearchCV object 'search_xgb' not found or not fitted. Skipping XGBoost importance plot.")

In [ ]:
import pandas as pd
import numpy as np
# Assuming 'best_xgb_model' is the fitted Pipeline object from GridSearchCV
# Assuming 'X_train' is the training feature DataFrame (before preprocessing in pipeline)
# 'interaction_df_final' or similar holds the original data used for X/y split

print("\n--- Defining Recommendation Function using XGBoost Model ---")

def recommend_universities_xgb(student_profile_dict, model_pipeline, candidate_universities, original_feature_columns, top_n=10):
    """
    Generates university recommendations for a student using a trained XGBoost pipeline.

    Args:
        student_profile_dict (dict): Dictionary containing the new student's features.
                                     Keys must match original feature columns before preprocessing.
        model_pipeline (Pipeline): The fitted Scikit-learn pipeline (preprocessor + XGBoost).
        candidate_universities (list): A list of university names to score.
        original_feature_columns (list): The list of feature column names expected by the
                                         pipeline's preprocessor (e.g., X_train.columns).
        top_n (int): The number of top recommendations to return.

    Returns:
        pandas.DataFrame: DataFrame with 'UniversityName' and 'AdmissionProbability',
                          sorted by probability, limited to top_n.
                          Returns empty DataFrame on error or no candidates.
    """
    if not candidate_universities:
        print("Warning: No candidate universities provided.")
        return pd.DataFrame({'UniversityName': [], 'AdmissionProbability': []})

    print(f"Generating predictions for {len(candidate_universities)} candidate universities...")

    # 1. Create input DataFrame for prediction
    # Repeat student profile for each candidate university
    student_df_list = []
    for university in candidate_universities:
        profile = student_profile_dict.copy()
        profile['UniversityName'] = university # Add the specific university name
        student_df_list.append(profile)

    input_df = pd.DataFrame(student_df_list)

    # Ensure columns match the order expected by the fitted preprocessor
    # Add missing columns as NaN if student_profile_dict was partial (might cause issues if not handled in preprocessor)
    try:
        input_df = input_df[original_feature_columns]
    except KeyError as e:
         print(f"Error: Input student profile is missing expected columns: {e}. Cannot predict.")
         print(f"Expected columns: {original_feature_columns}")
         print(f"Provided keys: {student_profile_dict.keys()}")
         return pd.DataFrame({'UniversityName': [], 'AdmissionProbability': []})


    # 2. Predict Probabilities using the full pipeline
    # The pipeline handles preprocessing internally before prediction
    try:
        print("Predicting probabilities...")
        probabilities = model_pipeline.predict_proba(input_df)
        # Probabilities array has shape (n_samples, n_classes), we want class 1 (Admitted)
        admission_probabilities = probabilities[:, 1]
        print("Prediction complete.")
    except Exception as e:
        print(f"Error during prediction: {e}")
        # Potentially print input_df.info() here for debugging
        return pd.DataFrame({'UniversityName': [], 'AdmissionProbability': []})


    # 3. Rank Universities
    results_df = pd.DataFrame({
        'UniversityName': candidate_universities,
        'AdmissionProbability': admission_probabilities
    })

    results_df = results_df.sort_values(by='AdmissionProbability', ascending=False)

    # 4. Return Top N
    return results_df.head(top_n)


# --- Example Usage ---

# ** PRE-REQUISITES **
# Ensure these exist from previous steps:
# - best_xgb_model: The fitted pipeline from GridSearchCV (search_xgb.best_estimator_)
# - X_train: The DataFrame used to train the model (needed for column names/order and candidate unis)

if 'best_xgb_model' in locals() and 'X_train' in locals():
    print("\n--- Generating Example Recommendations ---")

    # 1. Define a New Student Profile (Use exact column names from X_train)
    #    Use values *before* scaling/encoding. Fill all relevant features.
    new_student = {
        'Overall_GPA_4_Scale': 3.7, # Example GPA
        'Total Score (Language Exam)': 105.0, # Example TOEFL score
        'Language Exam': 'TOEFL',
        'Total Score (Competitive Exam)': 325.0, # Example GRE score
        'Competitive Exam': 'GRE',
        'Future Desired Degree': 'Masters',
        'Desired Field of Study': 'Computer Science', # Match categories used in training
        'Desired Specialization': 'Software Engineering' # Match categories used in training
        # Add any other features your model expects, e.g. UniversityName will be added in the function
    }
    print(f"New student profile: {new_student}")


    # 2. Get Candidate Universities (e.g., all unique universities seen in training)
    #    Using X_train['UniversityName'] avoids data leakage from test set
    try:
        candidate_universities = X_train['UniversityName'].unique().tolist()
        print(f"Number of candidate universities from training data: {len(candidate_universities)}")
    except KeyError:
         print("Error: 'UniversityName' column not found in X_train. Cannot get candidates.")
         candidate_universities = []


    # 3. Get Original Feature Columns Order (from X_train used to fit preprocessor)
    original_feature_columns = X_train.columns.tolist()


    # 4. Call the Recommendation Function
    if candidate_universities:
        recommendations_df = recommend_universities_xgb(
            student_profile_dict=new_student,
            model_pipeline=best_xgb_model,
            candidate_universities=candidate_universities,
            original_feature_columns=original_feature_columns,
            top_n=10 # Request top 10 recommendations
        )

        print("\n--- Top Recommended Universities ---")
        print(recommendations_df)
    else:
        print("\nCannot generate recommendations as no candidate universities were found.")

else:
    print("\nCould not run example: 'best_xgb_model' or 'X_train' not found in environment.")